In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr




In [2]:
load_dotenv()
MODEL = "gpt-4.1-mini"
client = OpenAI()


In [25]:
system_message ="""
You are chatBot for IRCTC . Keep your answers concise and to the point.
Your answers shoud be small one-liners.If you dont know the answer, just say "I don't know. if Asked to update the price, Check whether it already exists with help of tools,if already exists, dont update it"
"""

In [4]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = client.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

In [5]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [6]:
ticket_prices ={"hyderabad":"1000", "mumbai":"2000", "delhi":"3000","chennai":"4000","kolkata":"5000"}

def get_ticket_price(destination):
    print(f"Getting ticket price for {destination}")
    price = ticket_prices.get(destination.lower(),"unknown city price not found")
    return f"The ticket price for {destination} is {price}"
    

In [7]:
get_ticket_price("Mumbai")

Getting ticket price for Mumbai


'The ticket price for Mumbai is 2000'

In [8]:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [9]:
tools = [{"type": "function", "function": price_function}]

In [10]:
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Get the price of a return ticket to the destination city.',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalProperties': False}}}]

In [26]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = client.chat.completions.create(model=MODEL, messages=messages,tools=tools)
    
    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = client.chat.completions.create(model=MODEL, messages=messages,tools=tools)
    return response.choices[0].message.content


    

In [12]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            price_details = get_ticket_price(city)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
    return responses



In [13]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [14]:

get_price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

set_price_function = {
    "name": "set_ticket_price",
    "description": "Set the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
            "price": {"type": "integer", "description": "The price of the ticket"},
        },
        "required": ["destination_city", "price"],
        "additionalProperties": False
    }
}


In [15]:
tools = [{"type": "function", "function": get_price_function},{
    "type": "function",
    "function": set_price_function
}]

In [16]:
import sqlite3

In [17]:
DB = "prices.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)')
    conn.commit()

In [22]:
def get_ticket_price(city):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city=?', (city,))
        result = cursor.fetchone()
        return f"The price of the ticket to {city} is {result[0]}" if result else "Price not found for this city"
        
def set_ticket_price(city, price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city, price) VALUES (?, ?)', (city, price))
        conn.commit()
        return f"The price of the ticket to {city} has been set to {price}"

In [23]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            price_details = get_ticket_price(city)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "set_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            price = arguments.get('price')
            set_ticket_price(city, price)
            responses.append({
                "role": "tool",
                "content": f"The price of the ticket to {city} has been set to {price}",
                "tool_call_id": tool_call.id
            })
    return responses

In [27]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7875
* To create a public link, set `share=True` in `launch()`.
